特征工程：
1、随机森林特征筛选
用随机森林进行特征重要性评估的思想比较简单，主要是看每个特征在随机森林中的每棵树上做了多大的贡献，然后取平均值，最后比较不同特征之间的贡献大小。
贡献度的衡量指标包括：基尼指数（gini）、袋外数据（OOB）错误率作为评价指标来衡量。

2、IV值，全称是Information Value，中文意思是信息价值，或者信息量。
若IV在（-∞，0.02]区间，视为无预测力变量
若IV在（0.02，0.1]区间，视为较弱预测力变量
若IV在（0.1，+∞）区间，视为预测力可以，而实际应用中，也是保留IV值大于0.1的变量进行筛选。

In [1]:
import numpy as np
import math
import pandas as pd
from sklearn.utils.multiclass import type_of_target
from scipy import stats

In [2]:
#读入数据
data = pd.read_csv("data.csv",engine = 'python')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4754 entries, 0 to 4753
Data columns (total 90 columns):
Unnamed: 0                                    4754 non-null int64
custid                                        4754 non-null int64
trade_no                                      4754 non-null object
bank_card_no                                  4754 non-null object
low_volume_percent                            4752 non-null float64
middle_volume_percent                         4752 non-null float64
take_amount_in_later_12_month_highest         4754 non-null int64
trans_amount_increase_rate_lately             4751 non-null float64
trans_activity_month                          4752 non-null float64
trans_activity_day                            4752 non-null float64
transd_mcc                                    4752 non-null float64
trans_days_interval_filter                    4746 non-null float64
trans_days_interval                           4752 non-null float64
regional_mobility

In [3]:
#采用课时1的方法分析数据
dict1 = data.isnull().sum()  #统计缺失值个数  dict
for key in list(dict1.keys()):  #去除没有缺失值的键值对
    if dict1[key] == 0:
        del dict1[key]
for key in list(dict1.keys()): 
    if dict1[key] > 2000:
        del dict1[key]
        data.drop(key,axis = 1,inplace = True)

# 数值型数据均值填充
for key in dict1.keys():
    if data[key].dtype != object:
        data[key].fillna(data[key].mean(),inplace = True)
    else:
        print('object类型的缺失值和个数',key,dict1[key])

#对特殊的字段删除  如trad_no bank_card_no等对象类型的字段
data.drop(['trade_no'],axis = 1,inplace = True)
data.drop(['bank_card_no'],axis = 1,inplace = True)
data.drop(['Unnamed: 0'],axis = 1,inplace = True)
data.drop(['id_name'],axis = 1,inplace = True)
data.drop(['source'],axis = 1,inplace = True)
data.isnull().sum()


object类型的缺失值和个数 reg_preference_for_trad 2
object类型的缺失值和个数 id_name 276
object类型的缺失值和个数 latest_query_time 304
object类型的缺失值和个数 loans_latest_time 297


custid                                     0
low_volume_percent                         0
middle_volume_percent                      0
take_amount_in_later_12_month_highest      0
trans_amount_increase_rate_lately          0
trans_activity_month                       0
trans_activity_day                         0
transd_mcc                                 0
trans_days_interval_filter                 0
trans_days_interval                        0
regional_mobility                          0
repayment_capability                       0
is_high_user                               0
number_of_trans_from_2011                  0
first_transaction_time                     0
historical_trans_amount                    0
historical_trans_day                       0
rank_trad_1_month                          0
trans_amount_3_month                       0
avg_consume_less_12_valid_month            0
abs                                        0
top_trans_count_last_1_month               0
avg_price_

In [4]:
# 时间类型众数填充
data['latest_query_time'].fillna(data['latest_query_time'].mode()[0],inplace = True)
data['loans_latest_time'].fillna(data['loans_latest_time'].mode()[0],inplace = True)
print(data['latest_query_time'].isnull().sum())
data['loans_latest_time'].isnull().sum()

0


0

In [5]:
import seaborn as sns
from sklearn import preprocessing
#处理分析数据类型为非object  对数值型数据做归一化处理
min_max_scaler = preprocessing.MinMaxScaler()
for i in data.columns:
    if data[i].dtype == object:
        print(i)
    else:
        data[i] = (data[i]- data[i].min())/(data[i].max()-data[i].min())
data.head()

reg_preference_for_trad
latest_query_time
loans_latest_time


,custid,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,0.697138,0.01,0.99,0.000000,0.000019,0.488636,0.308370,0.375,0.094737,0.095652,...,0.29,0.244638,0.013777,0.862069,0.055556,0.10,0.004505,0.014493,0.038674,0.061538
1,0.133331,0.02,0.94,0.029412,0.000027,1.000000,0.468062,0.425,0.105263,0.043478,...,0.35,0.254783,0.173364,0.919540,0.277778,0.30,0.085586,0.113043,0.016575,0.012308
2,0.711603,0.04,0.96,0.000000,0.000021,1.000000,0.089207,0.275,0.238596,0.078261,...,0.16,0.181159,0.048220,1.000000,0.055556,0.05,0.015766,0.050725,0.011050,0.024615
3,0.451881,0.00,0.96,0.029412,0.000003,0.511364,0.819383,0.500,0.049123,0.008696,...,0.32,0.223333,0.187141,0.919540,0.277778,0.25,0.112613,0.147101,0.011050,0.018462
4,0.624214,0.01,0.99,0.000000,0.000010,1.000000,0.156388,0.275,0.231579,0.165217,...,0.23,0.236232,0.095293,0.908046,0.111111,0.10,0.031532,0.099638,0.066298,0.375385


In [6]:
#非数值型数据分析处理
data.reg_preference_for_trad.value_counts()
data = data.join(pd.get_dummies(data.reg_preference_for_trad))
data.drop(['reg_preference_for_trad'],axis = 1,inplace = True)


In [7]:

# 时间类型众数填充
data['latest_query_time'].fillna(data['latest_query_time'].mode()[0],inplace = True)
data['loans_latest_time'].fillna(data['loans_latest_time'].mode()[0],inplace = True)
print(data['latest_query_time'].isnull().sum())
data['loans_latest_time'].isnull().sum()
data['发生时间1'] = pd.to_datetime(data['latest_query_time'],format='%Y-%m-%d')#format='%Y%m%d%H%M%S'
data['发生时间2'] = pd.to_datetime(data['loans_latest_time'],format='%Y-%m-%d')#format='%Y%m%d%H%M%S'
data['发生时间2'].head()
data['年份1'] = [i.year for i in data['发生时间1']]
data['月份1'] = [i.month for i in data['发生时间1']]

data['年份2'] = [i.year for i in data['发生时间2']]
data['月份2'] = [i.month for i in data['发生时间2']]

cols = ['年份1','月份1','年份2','月份2']
for col in cols:
    data = data.join(pd.get_dummies(data[col]),how='left', lsuffix='_left', rsuffix='_right')
cols = ['年份1','月份1','年份2','月份2','发生时间1','发生时间2','latest_query_time','loans_latest_time']
for col in cols:
    data.drop([col],axis = 1,inplace = True)

0


In [8]:
#求woe值和iv值
def woe(X, y, event):
    res_woe = []  #列表存放woe字典
    res_iv = []  #列表存放iv
    X1 = feature_discretion(X) #对连续型特征进行处理
    for i in range(0, X1.shape[-1]):  #遍历所有特征
        x = X1[:, i]  #单个特征
        woe_dict, iv1 = woe_single_x(x, y, event)  #计算单个特征的woe值
        res_woe.append(woe_dict)
        res_iv.append(iv1)
    return np.array(res_woe), np.array(res_iv)  #返回数组

#求单个特征的woe值
def woe_single_x(x, y, event):
    event_total, non_event_total = count_binary(y, event) #计算好人坏人总数
    x_labels = np.unique(x) #特征中的分段
    woe_dict = {}  #存放每个分段的名称 以及 其对应的woe值
    iv = 0
    for x1 in x_labels: #遍历每个分段
        y1 = y[np.where(x == x1)[0]]
        event_count, non_event_count = count_binary(y1, event=event)
        rate_event = 1.0 * event_count / event_total
        rate_non_event = 1.0 * non_event_count / non_event_total
        #woe无穷大时处理
        if rate_event == 0:
            print()#print("{'",x1,"'}"+":全是好人") #只输出不做处理
        elif rate_non_event == 0:
            print()#print("{'",x1,"'}"+":全是坏人")
        else:
            woe1 = math.log(rate_event / rate_non_event)
            woe_dict[x1] = woe1  
            iv += (rate_event - rate_non_event) * woe1
    return woe_dict, iv

#计算个数
def count_binary(a, event):
    event_count = (a == event).sum()
    non_event_count = a.shape[-1] - event_count
    return event_count, non_event_count

#判断特征数据是否为离散型
def feature_discretion(X):
    temp = []
    for i in range(0, X.shape[-1]):
        x = X[:, i]
        x_type = type_of_target(x)
        if pd.Series(list(x)).dtype != 'O':
            x1 = discrete(x)
            temp.append(x1)
        else:
            temp.append(x)
    return np.array(temp).T

#对连续型特征进行离散化
def discrete(x):
    res = np.array([0] * x.shape[-1], dtype=int)
    for i in range(5):
        point1 = stats.scoreatpercentile(x, i * 20)
        point2 = stats.scoreatpercentile(x, (i + 1) * 20)
        x1 = x[np.where((x >= point1) & (x <= point2))]
        mask = np.in1d(x, x1)
        res[mask] = (i + 1)
    return res

In [9]:
y = data['status'].values
x = data.drop('status', axis=1).values
a, b = woe(x, y, 1)  #计算IV值

In [10]:
name = data.drop('status', axis=1).columns

In [11]:
dic = dict(zip(name, b))
dic_sort= sorted(dic.items(),key = lambda x:x[1],reverse = True)

In [12]:
dic_sort

[('trans_fail_top_count_enum_last_1_month', 0.5434151479097552),
 ('history_fail_fee', 0.49307981436539067),
 ('loans_score', 0.4228985392642032),
 ('apply_score', 0.34200642990772334),
 ('loans_overdue_count', 0.33542042413754747),
 ('trans_fail_top_count_enum_last_12_month', 0.24514948545819687),
 ('trans_fail_top_count_enum_last_6_month', 0.2401158488566566),
 ('latest_one_month_fail', 0.1469171942527224),
 ('latest_one_month_suc', 0.1321532086641229),
 ('max_cumulative_consume_later_1_month', 0.10619295459906644),
 ('rank_trad_1_month', 0.10450572317649498),
 ('trans_top_time_last_1_month', 0.08579637244602582),
 ('trans_day_last_12_month', 0.0765813508883087),
 ('top_trans_count_last_1_month', 0.06484703702280309),
 ('pawns_auctions_trusts_consume_last_1_month', 0.05664026697293398),
 ('consfin_avg_limit', 0.0507326032363208),
 ('trans_amount_3_month', 0.03749800010580736),
 ('consfin_credit_limit', 0.037043251278019135),
 ('consume_top_time_last_1_month', 0.03640380948863156),
 (

2.利用随机森林进行变量选择
主要根据模型输出的feature_importance进行变量选择

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
x = data.drop('status', axis=1)
y = data['status']

In [15]:
rf = RandomForestClassifier()
rf.fit(x, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [16]:
fm = rf.feature_importances_
dic_rf = dict(zip(x.columns, fm))
dic_sort_rf = sorted(dic_rf.items(), key=lambda x: x[1], reverse=True)

In [17]:
dic_sort_rf

[('trans_fail_top_count_enum_last_1_month', 0.05427548385510853),
 ('history_fail_fee', 0.041288751477839385),
 ('loans_score', 0.03154222553301346),
 ('loans_overdue_count', 0.02874058555524992),
 ('apply_score', 0.021160186693651366),
 ('historical_trans_amount', 0.01811663712852164),
 ('max_cumulative_consume_later_1_month', 0.01803447690017085),
 ('latest_one_month_fail', 0.017867288779913615),
 ('trans_amount_3_month', 0.017031096476499703),
 ('avg_price_last_12_month', 0.016668984278347697),
 ('repayment_capability', 0.016089107040234762),
 ('trans_amount_increase_rate_lately', 0.01567005449394846),
 ('pawns_auctions_trusts_consume_last_6_month', 0.01561007066262404),
 ('loans_long_time', 0.015025972790604487),
 ('consfin_avg_limit', 0.01452210231238476),
 ('trans_day_last_12_month', 0.014222333019692312),
 ('trans_activity_day', 0.014120300125616717),
 ('abs', 0.013905019062709498),
 ('historical_trans_day', 0.01388710935388814),
 ('custid', 0.013844543553303487),
 ('loans_settl

3.采用互信息进行变量选择

In [18]:
from sklearn.feature_selection import mutual_info_classif

In [19]:
mf = mutual_info_classif(x, y)

In [20]:
dic_mf = dict(zip(x.columns, mf))
dic_sort_mf = sorted(dic_mf.items(), key=lambda x:x[1], reverse=True)

In [21]:
dic_sort_mf

[('trans_fail_top_count_enum_last_1_month', 0.05942519541323099),
 ('loans_score', 0.0592567947751399),
 ('history_fail_fee', 0.054295280735540574),
 ('latest_one_month_fail', 0.045157999612830624),
 ('apply_score', 0.04255691824009755),
 ('loans_overdue_count', 0.03528574401972984),
 ('trans_fail_top_count_enum_last_12_month', 0.021529649141905693),
 ('trans_fail_top_count_enum_last_6_month', 0.018861190468745903),
 ('middle_volume_percent', 0.01484032464900853),
 ('rank_trad_1_month', 0.01371044095121654),
 ('trans_top_time_last_1_month', 0.013074692268285837),
 ('historical_trans_day', 0.012821871465464652),
 ('max_cumulative_consume_later_1_month', 0.012095888693473),
 ('query_org_count', 0.010701026445428807),
 ('latest_one_month_loan', 0.0099593808182048),
 ('railway_consume_count_last_12_month', 0.009878125628330237),
 ('consfin_org_count_current', 0.009669753715160878),
 ('trans_day_last_12_month', 0.009659559808984142),
 ('pawns_auctions_trusts_consume_last_6_month', 0.0096056